# A messy comparison between source JSON and composited CSVs ...
This performs some data integrity checks, loading the JSON files into pandas dataframes and then comparing the summary statistics of the columns that should align with the extract-to-csvs output. By default it's set up to compare only reason code F, which is busy enough and contains a lot of variety.

This looks for the extracted F file in the root directory of the project now.

Rosie, @cimar, notes that extra cleaning on her end solved the vast majority of points of conflict in her initial effort, with data through mid-March. Exported for manual review. (Do the aggregated columns make sense to aggregate? Notes on the persistent conflicts.)

In [1]:
import pandas,os
from tqdm import tqdm
# SAVEDIR = "data/cancelled_contracts"

SAVEDIR = "data/"
os.makedirs(SAVEDIR, exist_ok=True)

## Mike isn't manually catching these and I don't know why he doesn't seem to have to
NA_STRS = ["N/A","NULL"]

dfs=[]
             
for file in tqdm(os.listdir(SAVEDIR)):
    if file[-4:] == "json":
        reason = file[-6]
        ndf = pandas.read_json(SAVEDIR+"/"+file)
        ndf["filedate"] = file.split("contracts-")[-1].split("_")[0]
        ndf["filename"] = file
        ndf["reason_code"] = reason
        dfs.append(ndf)

df = pandas.concat(dfs)
df = df.replace(NA_STRS, None)
df.describe().transpose().head(50)

fdf = df[df["reason_code"]=="F"]
fdf["filedate"] = pandas.to_datetime(fdf["filedate"])
fdf["relevantContractDates__effectiveDate"] = fdf[["content__award__relevantContractDates__effectiveDate",
                                                   "content__IDV__relevantContractDates__effectiveDate",
                                                   "content__OtherTransactionAward__contractDetail__relevantContractDates__effectiveDate"]].fillna('').sum(axis=1)

# rosief = fdf[fdf["filedate"]<="2025-03-16"]   # Limit files to a particular date?
# Or just grab all the latest?

rosief = fdf

100%|██████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 166440.63it/s]


ValueError: No objects to concatenate

In [ ]:
mikef = pandas.read_csv("collected_F.csv")
# mikef = mikef[mikef["filedate"]<="2025-03-20"]
mikef_cols = pandas.Series(mikef.columns)
mikef_cols_temp = mikef_cols[mikef_cols.str.contains("placeOfPerformance")]
mikef.groupby("contract_type")[mikef_cols_temp].nunique()#.describe().head().transpose().head(50)#["dollarValues__obligatedAmount"].sum()

In [ ]:
## Cleaning for better alignment!

def summary_sum(s):
    try:
        pandas.to_numeric(s).sum()
    except:
        return 0

def agg(row,cols,method):
    row[cols].agg(method)

def count_nunique(cols,df):
    ls = []
    for col in cols:
        ls += df[col].to_list()
    return pandas.Series(ls).nunique()


## Maybe could have avoided all of this by making them all strings to start out with
def make_str_coerce_floats_to_int(messy_str,zfill=None):
    if pandas.isna(messy_str):
        return None
    else:
        temp = str(messy_str).split(".")[0]
        if zfill:
            return temp.zfill(zfill)
        return temp
    
rosief["content__award__awardID__awardContractID__agencyID"] = rosief["content__award__awardID__awardContractID__agencyID"].apply(make_str_coerce_floats_to_int)
mikef["awardContractID__agencyID"] = mikef["awardContractID__agencyID"].apply(make_str_coerce_floats_to_int)

rosief[['content__award__purchaserInformation__contractingOfficeAgencyID',
        'content__IDV__purchaserInformation__contractingOfficeAgencyID',
        'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID']] = rosief[['content__award__purchaserInformation__contractingOfficeAgencyID',
                                                                                                                      'content__IDV__purchaserInformation__contractingOfficeAgencyID',
                                                                                                                      'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID']].map(make_str_coerce_floats_to_int)
mikef["contractingOfficeAgencyID"] = mikef["contractingOfficeAgencyID"].apply(make_str_coerce_floats_to_int)

rosief[['content__award__purchaserInformation__contractingOfficeAgencyID__departmentID',
        'content__IDV__purchaserInformation__contractingOfficeAgencyID__departmentID',
        'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID__departmentID']] = rosief[['content__award__purchaserInformation__contractingOfficeAgencyID__departmentID',
                                                                                                                                    'content__IDV__purchaserInformation__contractingOfficeAgencyID__departmentID',
                                                                                                                                    'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID__departmentID']].map(make_str_coerce_floats_to_int)
mikef['contractingOfficeAgencyID__departmentID'] = mikef['contractingOfficeAgencyID__departmentID'].apply(make_str_coerce_floats_to_int)

rosief["content__award__placeOfPerformance__placeOfPerformanceZIPCode"] = rosief["content__award__placeOfPerformance__placeOfPerformanceZIPCode"].apply(lambda x: make_str_coerce_floats_to_int(x,zfill=9))
rosief["content__OtherTransactionAward__contractDetail__placeOfPerformance__placeOfPerformanceZIPCode"] = rosief["content__OtherTransactionAward__contractDetail__placeOfPerformance__placeOfPerformanceZIPCode"].apply(lambda x: make_str_coerce_floats_to_int(x,zfill=9))
mikef["placeOfPerformanceZIPCode"] = mikef["placeOfPerformanceZIPCode"].apply(lambda x: make_str_coerce_floats_to_int(x,zfill=9))

rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__ZIPCode',
         'content__IDV__vendor__vendorSiteDetails__vendorLocation__ZIPCode',
         'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__ZIPCode']] = rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__ZIPCode',
                                                                                                                             'content__IDV__vendor__vendorSiteDetails__vendorLocation__ZIPCode',
                                                                                                                             'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__ZIPCode']].map(make_str_coerce_floats_to_int,zfill=9)

mikef["vendorLocation__ZIPCode"] = mikef["vendorLocation__ZIPCode"].apply(lambda x: make_str_coerce_floats_to_int(x,zfill=9))

rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__phoneNo',
         'content__IDV__vendor__vendorSiteDetails__vendorLocation__phoneNo',
         'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__phoneNo']] = rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__phoneNo',
                                                                                                                         'content__IDV__vendor__vendorSiteDetails__vendorLocation__phoneNo',
                                                                                                                         'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__phoneNo']].map(make_str_coerce_floats_to_int)
mikef["vendorLocation__phoneNo"] = mikef["vendorLocation__phoneNo"].apply(make_str_coerce_floats_to_int)


In [ ]:
def list_vals(coln,df2):
    df2_cols = pandas.Series(df2.columns)
    relevant = df2_cols[df2_cols.str.endswith(coln)]
    rel_ls = relevant.to_list()
    return rel_ls
    
mikef_col_df = pandas.DataFrame(mikef_cols)
mikef_col_df.columns = ["column_name"]
mikef_col_df["aggregated_cols"] = mikef_col_df["column_name"].apply(lambda x: list_vals(x,rosief))
mikef_col_df["count_agg_cols"] = mikef_col_df["aggregated_cols"].apply(len)
mikef_col_df[mikef_col_df["count_agg_cols"]==4]["aggregated_cols"].loc[8]

mikef_col_df["mike_count"] = mikef_col_df["column_name"].apply(lambda x: mikef[x].count())
mikef_col_df["mike_sum"] = mikef_col_df["column_name"].apply(lambda x: summary_sum(x))
mikef_col_df["mike_nunique"] = mikef_col_df["column_name"].apply(lambda x: mikef[x].nunique())

mikef_col_df["rosie_count"] = mikef_col_df.apply(lambda x: rosief[x["aggregated_cols"]].count().sum(),axis=1)
mikef_col_df["rosie_sum"] = mikef_col_df.apply(lambda x: rosief[x["aggregated_cols"]].apply(summary_sum).sum(),axis=1)
mikef_col_df["rosie_nunique"] = mikef_col_df["aggregated_cols"].apply(lambda x: count_nunique(x,rosief))

mikef_col_df["counts_match"] = mikef_col_df["mike_count"] == mikef_col_df["rosie_count"]
mikef_col_df["sums_match"] = mikef_col_df["mike_sum"] == mikef_col_df["rosie_sum"]
mikef_col_df["nunique_match"] = mikef_col_df["mike_nunique"] == mikef_col_df["rosie_nunique"]

mikef_col_df.to_csv("column_review.csv")

In [ ]:
rosief["content__award__awardID__awardContractID__agencyID"] = rosief["content__award__awardID__awardContractID__agencyID"].apply(make_str_coerce_floats_to_int)
mikef["awardContractID__agencyID"] = mikef["awardContractID__agencyID"].apply(make_str_coerce_floats_to_int)

mikef_col_df[~mikef_col_df["counts_match"]]#["aggregated_cols"].iloc[-1]

# The rest of this is junk/scratch notes!

In [ ]:
rosie_zips = pandas.concat([rosief["content__award__placeOfPerformance__placeOfPerformanceZIPCode"],rosief["content__OtherTransactionAward__contractDetail__placeOfPerformance__placeOfPerformanceZIPCode"]])
rosie_zips.value_counts()

temp = pandas.concat([mikef["placeOfPerformanceZIPCode"].value_counts(),rosie_zips.value_counts()], axis=1)
temp.columns = ["m","r"]
temp[temp["m"]!=temp["r"]]

In [ ]:

#df['combined'] = df['col1'].fillna(df['col2'])

rosief["combine_zip"] = rosief['content__award__placeOfPerformance__placeOfPerformanceZIPCode'].fillna(rosief['content__OtherTransactionAward__contractDetail__placeOfPerformance__placeOfPerformanceZIPCode'])
rosief["temp_zip_col"] = rosief["combine_zip"].apply(lambda x: str(x).split(".")[0]).str.zfill(9)
mikef["temp_zip_col"] = mikef["placeOfPerformanceZIPCode"].apply(lambda x: str(x).split(".")[0]).str.zfill(9)

zip_temp = pandas.concat([rosief["temp_zip_col"].value_counts(),mikef["temp_zip_col"].value_counts()],axis=1)
zip_temp.columns = ["rosie_count","mike_count"]
zip_temp[zip_temp["rosie_count"]!=zip_temp["mike_count"]]

In [ ]:
mikef_col_df[~mikef_col_df["nunique_match"]]["aggregated_cols"].iloc[-1]

In [ ]:
rosief["temp_contract_off_id"] = rosief[['content__award__purchaserInformation__contractingOfficeAgencyID',
                                      'content__IDV__purchaserInformation__contractingOfficeAgencyID',
                                      'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID']].fillna("").astype(str).sum(axis=1)

rosief["temp_contract_off_id"] = rosief["temp_contract_off_id"].apply(make_str_coerce_floats_to_int)
rosief.groupby("contract_type")["temp_contract_off_id"].nunique()

In [ ]:
rosief["temp_contract_off_id"] = rosief["temp_contract_off_id"].apply(make_str_coerce_floats_to_int)
mikef.groupby("contract_type")["contractingOfficeAgencyID"].nunique()

In [ ]:
mikef[mikef["contractingOfficeAgencyID"].isin(rosief["temp_contract_off_id"])==False]["contractingOfficeAgencyID"].value_counts()

In [ ]:
mikef[mikef["contractingOfficeAgencyID"]=="4732"].head().transpose().head(50)[0].title

In [ ]:
TITLE = 'DELIVERY ORDER 47QSSC25F3UL7 (P00001) awarded to JLWS ENTERPRISES INCORPORATED, was modified for the amount of -$804.05'

rosief[(rosief["filename"]=="contracts-2025-01-20_F.json")&(rosief["title"]==TITLE)]["temp_contract_off_id"]

In [ ]:
4732 in rosief["temp_contract_off_id"]

In [ ]:
rosief["temp_contract_off_id"].value_counts()

In [ ]:
rosief.groupby("contract_type")[['content__award__vendor__vendorSiteDetails__vendorLocation__state',
 'content__IDV__vendor__vendorSiteDetails__vendorLocation__state',
 'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__state']].count()

rosief["temp_vendor_state"] = rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__state',
 'content__IDV__vendor__vendorSiteDetails__vendorLocation__state',
 'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__state']].fillna('').sum(axis=1)

rosief["temp_vendor_state"].value_counts()

state_temp = pandas.concat([rosief["temp_vendor_state"].value_counts(),mikef["vendorLocation__state"].value_counts()],axis=1)
state_temp.columns = ["rosie_count","mike_count"]
state_temp[state_temp["rosie_count"]!=state_temp["mike_count"]]

In [ ]:
mikef["content__award__contractData__solicitationID"].unique()

In [ ]:
rosief["content__award__contractData__solicitationID"].unique()